In [2]:
def create_circle(m,r,ls='-'):
    return patches.Circle((m,0), radius=r, angle=180.,lw=2,fill=False,ls=ls)

In [3]:
def principal_to_pq(s_1fe,s_3fe):
    return (s_1fe + s_3fe)/2, (s_1fe - s_3fe)/2

In [4]:
def find_b_tanalpha(s_1fe,s_3fe):
    s_m, t_m = principal_to_pq(s_1fe,s_3fe)
    regr = sp.stats.linregress(s_m,t_m)
    return regr.slope, regr.intercept

In [5]:
def find_c_sinphi(s_1fe,s_3fe):
    s_m, t_m = principal_to_pq(s_1fe,s_3fe)
    regr = sp.stats.linregress(s_m,t_m)
    return regr.slope, regr.intercept/np.cos(np.arcsin(regr.slope))

In [6]:
def find_failure_stresses(s_1fe,s_3fe):
    sinphi, c = find_c_sinphi(s_1fe,s_3fe)
    phi = np.arcsin(sinphi)
    theta = np.pi/4 + phi/2
    tau_alpha = (c + s_3fe*np.tan(phi))/(1-np.tan(phi)/np.tan(theta))
    s_alpha = (tau_alpha - c)/np.tan(phi)
    return s_alpha, tau_alpha

In [7]:
def plot_basic_circles(s_1fe,s_3fe):
    fig, ax = plt.subplots(figsize=(12,12))
    s_m, t_m = principal_to_pq(s_1fe,s_3fe)
    for i in range(len(s_1fe)):
        ax.add_patch(create_circle(s_m[i],t_m[i]))

    ax.set(aspect='equal')
    ax.set_xlabel("$\\sigma'_\\mathrm{n}$ / kPa")
    ax.set_ylabel('$\\tau$ / kPa')
    #ax.legend(fontsize=12)
    ax.set_xlim(0,np.max(s_1fe))
    ax.set_ylim(0,np.max(s_1fe)/2)
    #ax.spines['left'].set_position(('data', 0))
    fig.tight_layout()
    plt.show();

In [8]:
def plot_pq(s_1fe,s_3fe,u=0):
    s_m, t_m = principal_to_pq(s_1fe,s_3fe)
    m, b = find_b_tanalpha(s_1fe,s_3fe)
    
    fig, ax = plt.subplots(figsize=(12,8))
    ax.set_xlabel("$\\sigma_\\mathrm{M}$ / kPa")
    ax.set_ylabel("$(\\sigma_1 - \\sigma_3) / 2$ / kPa")
    ax.plot(s_m,t_m,ls='',marker='d',label="$\\sigma_\\mathrm{M} = (\\sigma_1'+\\sigma_3')/2$")
    
    ax.plot([0,np.max(s_m)],[b,b+m*np.max(s_m)],label="$b' + \\sigma_\\mathrm{M}' \\tan \\alpha'$")
    fig.tight_layout()
    ax.legend()
    ax.set_xlim(0,np.max(s_m)*1.02)
    ax.set_ylim(0,np.max(t_m)*1.02)
    plt.show();

In [9]:
def plot_circles_tangent(s_1fe,s_3fe):
    fig, ax = plt.subplots(figsize=(12,12))
    s_m, t_m = principal_to_pq(s_1fe,s_3fe)
    sinphi, c = find_c_sinphi(s_1fe,s_3fe)
    for i in range(len(s_1fe)):
        ax.add_patch(create_circle(s_m[i],t_m[i]))

    ax.plot([0,np.max(s_1fe)],[c,c+np.max(s_1fe)*np.tan(np.arcsin(sinphi))],label="$c' + \\sigma_\\mathrm{n}' \\tan \\varphi'$")
    ax.set(aspect='equal')
    ax.set_xlabel("$\\sigma'_\\mathrm{n}$ / kPa")
    ax.set_ylabel('$\\tau$ / kPa')
    ax.legend()
    ax.set_xlim(0,np.max(s_1fe))
    ax.set_ylim(0,np.max(s_1fe)/2)
    #ax.spines['left'].set_position(('data', 0))
    fig.tight_layout()
    plt.show();

In [10]:
def plot_circles_failure(s_1fe,s_3fe):
    fig, ax = plt.subplots(figsize=(12,12))
    s_m, t_m = principal_to_pq(s_1fe,s_3fe)
    sinphi, c = find_c_sinphi(s_1fe,s_3fe)
    s_a, t_a = find_failure_stresses(s_1fe,s_3fe)
    for i in range(len(s_1fe)):
        ax.add_patch(create_circle(s_m[i],t_m[i]))

    ax.plot([0,np.max(s_1fe)],[c,c+np.max(s_1fe)*np.tan(np.arcsin(sinphi))],label="$c' + \\sigma_\\mathrm{n}' \\tan \\varphi'$")
    ax.plot(s_a,t_a,ls='',marker='o', label="$\\sigma_\\vartheta', \\tau_\\vartheta$",color='red')
    for i in range(len(s_1fe)):
        ax.plot([s_3fe[i],s_a[i]],[0,t_a[i]],ls='--',color='red')
    ax.set(aspect='equal')
    ax.set_xlabel("$\\sigma'_\\mathrm{n}$ / kPa")
    ax.set_ylabel('$\\tau$ / kPa')
    ax.legend()
    ax.set_xlim(0,np.max(s_1fe))
    ax.set_ylim(0,np.max(s_1fe)/2)
    #ax.spines['left'].set_position(('data', 0))
    fig.tight_layout()
    plt.show();

In [11]:
def plot_pq_tot(s_1fe,s_3fe,u=0,u0=0):
    s_me, t_me = principal_to_pq(s_1fe,s_3fe)
    m, b = find_b_tanalpha(s_1fe,s_3fe)
    
    s_conf = s_3fe+u
    
    fig, ax = plt.subplots(figsize=(12,8))
    ax.set_xlabel("$\\sigma_\\mathrm{M}$ / kPa")
    ax.set_ylabel("$(\\sigma_1 - \\sigma_3) / 2$ / kPa")
    ax.plot(s_me,t_me,ls='',marker='d',label="$\\sigma_\\mathrm{M} = (\\sigma_1'+\\sigma_3')/2$")
    ax.plot(s_me+u,t_me,ls='',marker='o',label="$\\sigma_\\mathrm{M} = (\\sigma_1+\\sigma_3)/2$")
    ax.plot(s_3fe+u,s_3fe*0,ls='',marker='s',label="$\\sigma_\\mathrm{M} = \\sigma_\\mathrm{c} + u_0 = \\sigma_3$")
    
    ax.plot([0,np.max(s_me)],[b,b+m*np.max(s_me)],label="$b' + \\sigma_\\mathrm{M}' \\tan \\alpha'$")
    
    
    for i in range(len(s_1fe)):
        t = np.linspace(0,t_me[i],40)
        du = np.power(np.linspace(0,u[i]-u0[i],40)/(u[i]-u0[i]+1.e-14),3)*(u[i]-u0[i]) + u0[i]
        ds = np.linspace(s_3fe[i]+u[i],s_me[i]+u[i],40)
        if (i==0):
            ax.plot([0,s_3fe[i]+u[i],s_me[i]+u[i]],[0,0,t_me[i]],color='green',label='Totalspannungspfad')
            ax.plot(ds-du,t,color='red',label='Effektivpannungspfad',ls='--')
        else:
            ax.plot([0,s_3fe[i]+u[i],s_me[i]+u[i]],[0,0,t_me[i]],color='green')
            ax.plot(ds-du,t,color='red',ls='--')
        for j in range(len(t)):
            if (np.abs(du[j])>10):
                ax.annotate("", xy=(ds[j]-du[j], t[j]), xytext=(ds[j], t[j]), arrowprops=dict(arrowstyle="->",color='blue'))
            else:
                ax.annotate("", xy=(ds[j]-du[j], t[j]), xytext=(ds[j], t[j]), arrowprops=dict(arrowstyle="-",color='blue'))
    
    fig.tight_layout()
    ax.set_xlim(0,np.max(s_me+u)*1.02)
    ax.set_ylim(0,np.max(t_me)*1.02)
    ax.legend()
    plt.show();

In [ ]:
def plot_circles_tangent_total(s_1fe,s_3fe,uf):
    fig, ax = plt.subplots(figsize=(12,12))
    s_m, t_m = principal_to_pq(s_1fe,s_3fe)
    sinphi, c = find_c_sinphi(s_1fe,s_3fe)
    for i in range(len(s_1fe)):
        ax.add_patch(create_circle(s_m[i],t_m[i]))
    
    s_mt, t_mt = principal_to_pq(s_1fe+uf,s_3fe+uf)
    sinphiu, cu = find_c_sinphi(s_1fe+uf,s_3fe+uf)
    for i in range(len(s_1fe)):
        ax.add_patch(create_circle(s_mt[i],t_mt[i],':'))

    ax.plot([0,np.max(s_1fe)],[c,c+np.max(s_1fe)*np.tan(np.arcsin(sinphi))],label="$c' + \\sigma_\\mathrm{n}' \\tan \\varphi'$")
    ax.plot([0,np.max(s_1fe+uf)],[cu,cu+np.max(s_1fe+uf)*np.tan(np.arcsin(sinphiu))],label="$c_\\mathrm{u} + \\sigma_\\mathrm{n} \\tan \\varphi_\\mathrm{u}$")
    ax.set(aspect='equal')
    ax.set_xlabel("$\\sigma'_\\mathrm{n}$, $\\sigma_\\mathrm{n}$ / kPa")
    ax.set_ylabel('$\\tau$ / kPa')
    ax.legend()
    ax.set_xlim(0,np.max(s_1fe+uf))
    ax.set_ylim(0,np.max(s_1fe+uf)/2)
    #ax.spines['left'].set_position(('data', 0))
    fig.tight_layout()
    plt.show();